In [7]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import  train_test_split 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout ,Input
from tensorflow.keras.applications import imagenet_utils
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import cv2
import progressbar
import os

In [8]:
imagePaths=[]

In [10]:
for i in os.listdir("/content/drive/MyDrive/dataset-train/dataset/"):
  for image in os.listdir("/content/drive/MyDrive/dataset-train/dataset/" + i + "/"):
    path= "/content/drive/MyDrive/dataset-train/dataset/" + i + "/" + image
    imagePaths.append(path)

In [ ]:
imagePaths

['./dataset/without_mask/67.jpg',
 './dataset/without_mask/53.jpg',
 './dataset/without_mask/229.jpg',
 './dataset/without_mask/augmented_image_130.jpg',
 './dataset/without_mask/388.jpg',
 './dataset/without_mask/359.jpg',
 './dataset/without_mask/231.jpg',
 './dataset/without_mask/39.jpg',
 './dataset/without_mask/322.jpg',
 './dataset/without_mask/341.jpg',
 './dataset/without_mask/augmented_image_78.jpg',
 './dataset/without_mask/augmented_image_76.jpg',
 './dataset/without_mask/50.jpg',
 './dataset/without_mask/332.jpg',
 './dataset/without_mask/augmented_image_42.jpg',
 './dataset/without_mask/172.jpg',
 './dataset/without_mask/augmented_image_278.jpg',
 './dataset/without_mask/augmented_image_163.jpg',
 './dataset/without_mask/augmented_image_292.jpg',
 './dataset/without_mask/198.jpg',
 './dataset/without_mask/266.jpg',
 './dataset/without_mask/augmented_image_99.jpg',
 './dataset/without_mask/302.jpg',
 './dataset/without_mask/20.jpg',
 './dataset/without_mask/328.jpg',
 './da

In [11]:
data = []
labels = []

In [12]:
for i in imagePaths:
  image = load_img(i , target_size=(224,224))
  image = img_to_array(image)
  image = imagenet_utils.preprocess_input(image)
  

  label = i.split("/")[-2]

  data.append(image)
  labels.append(label)


In [13]:
len(data)

1376

In [14]:
len(labels)

1376

In [15]:
data =np.array(data)
labels = np.array(labels)

In [16]:
lb = LabelBinarizer()
Labels = lb.fit_transform(labels)

In [17]:
Labels

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [18]:
Labels = to_categorical(Labels)

In [19]:
Labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [20]:
augment = ImageDataGenerator(rotation_range=30, height_shift_range=0.1, zoom_range=0.1 ,horizontal_flip=True, fill_mode="nearest", width_shift_range=0.1)

In [21]:
basemodel = VGG19(include_top=False , input_tensor=Input(shape=(224,224,3)))

80142336/80134624 [==============================] - 0s 0us/step


In [22]:
headmodel = basemodel.output

In [23]:
headmodel = Flatten(name="flatten" )(headmodel)
headmodel= Dense(256 , activation="relu")(headmodel)
headmodel = Dropout(0.5)(headmodel)

headmodel = Dense(2 , activation="softmax" )(headmodel)

model = Model(inputs=basemodel.input , outputs = headmodel)

In [24]:
for layer in basemodel.layers:
  layer.trainable = False

In [25]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [26]:
trainX , testX , trainY , testY  = train_test_split(data , Labels , test_size=0.2)

In [27]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [28]:
print("Model is going to Train.......")
H = model.fit(augment.flow(trainX, trainY, batch_size=32), validation_data=(testX, testY), epochs=7)

Model is going to Train.......
Epoch 1/7
35/35 [==============================] - 53s 1s/step - loss: 4.3350 - accuracy: 0.8660 - val_loss: 0.5612 - val_accuracy: 0.9855
Epoch 2/7
35/35 [==============================] - 52s 1s/step - loss: 1.8770 - accuracy: 0.9800 - val_loss: 0.8466 - val_accuracy: 0.9855
Epoch 3/7
35/35 [==============================] - 52s 1s/step - loss: 1.3169 - accuracy: 0.9829 - val_loss: 0.2229 - val_accuracy: 0.9928
Epoch 4/7
35/35 [==============================] - 52s 1s/step - loss: 0.2907 - accuracy: 0.9953 - val_loss: 0.1249 - val_accuracy: 0.9964
Epoch 5/7
35/35 [==============================] - 52s 1s/step - loss: 0.5878 - accuracy: 0.9905 - val_loss: 0.3698 - val_accuracy: 0.9928
Epoch 6/7
35/35 [==============================] - 51s 1s/step - loss: 0.2581 - accuracy: 0.9952 - val_loss: 0.5735 - val_accuracy: 0.9928
Epoch 7/7
35/35 [==============================] - 51s 1s/step - loss: 0.1763 - accuracy: 0.9907 - val_loss: 0.4956 - val_accuracy: 0.9

In [31]:
model.save("mask_detection.h5")

In [32]:
from google.colab import files
files.download("mask_detection.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

In [34]:
from tensorflow.keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

In [35]:
model = load_model("mask_detection.h5")

In [36]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [37]:
face_class = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [ ]:
cam = cv2.VideoCapture(0)
#cam = cv2.VideoCapture("http://192.168.1.2:8080/video")
while True:
    _ , img = cam.read()
    gray = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
    faces = face_class.detectMultiScale(gray , 1.3 , 5)
    for(x,w,y,h) in faces:
        face = img[y:y+h , x:x+w]
        face = cv2.resize(face , (224,224))
        face = img_to_array(face)
        face = np.expand_dims(face , axis=0)
        face= preprocess_input(face)
        
        mask , no_mask = model.predict(face)[0]
        if mask > no_mask:
            print("Mask_detected")
        else:
            print("Mask not detected")
        img=cv2.resize(img , (700 , 400))   
        cv2.imshow("My Face", img)
    if cv2.waitKey(1) == 13:
        break
cam.release()
cv2.destroyAllWindows()